In [1]:
import pandas as pd
from rich import print as rprint
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url, dtype_backend='pyarrow', engine='pyarrow')
city_mpg = df.city08
highway_mpg = df.highway08
len(dir(city_mpg))

420

`.apply` applies function to an entire series or element-wise to every value. If it does the latter, you lose the optimized and fast storage of `numpy` because you pull data into Python. It breaks out of the fast vectorized code paths we can leverage in pandas

In [24]:
%%timeit
def gt20(val):
    return val > 20

city_mpg.apply(gt20)


2.54 ms ± 28.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Use the broadcasted `.gt` method, and it runs almost 50 times faster

In [17]:
%timeit city_mpg.gt(20)

27.4 μs ± 295 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Show the top five makes and label everything else *other*

In [18]:
make = df.make
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [19]:
make.value_counts()

make
Chevrolet                           4003
Ford                                3371
Dodge                               2583
GMC                                 2494
Toyota                              2071
                                    ... 
Grumman Allied Industries              1
Environmental Rsch and Devp Corp       1
General Motors                         1
Goldacre                               1
Isis Imports Ltd                       1
Name: count, Length: 136, dtype: int64[pyarrow]

In [21]:
top5 = make.value_counts().index[:5]
rprint(top5)

Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota'], dtype='string[pyarrow]', name='make')

In [25]:
%%timeit
def generalize_top5(val):
    if val in top5:
        return val
    return 'Other'
make.apply(generalize_top5)

18.9 ms ± 335 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
rprint(make.where(make.isin(top5), other='Other'))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: string[pyarrow]

In [31]:
%timeit make.where(make.isin(top5),other='Other')

732 μs ± 10.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The complement of the `where` method is the `mask` method. Wherever the condition is `False` it keeps the original values; if it is `True` it replaces the value with the `other` parameter. The tilde, `~`, performs an inversion of the boolean array, switching all `True` values to `False` and vice versa

In [34]:
make.mask(~make.isin(top5),other='Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: string[pyarrow]

Broadcast `numpy` function to entire series. `np.log` is a universal function ( *ufunc* ) and works on arrays.

In [37]:
import numpy as np

np.__version__

%timeit city_mpg.apply(np.log)

125 μs ± 687 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Compare that with Python's math module:

In [38]:
import math
%timeit city_mpg.apply(math.log)

2.29 ms ± 6.79 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


keep the top five makes and get the top 10 for the remainder of the top ten, other for the rest

In [39]:
vc = make.value_counts()
top5 = vc.index[:5] # start at 0 stop at 5
top10 = vc.index[:10] # start at 0 stop at 10

def generalize(val):
    if val in top5:
        return val
    elif val in top10:
        return 'Top10'
    else:
        return 'Other'
    
make.apply(generalize)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

### `.case_when` method
replace values where conditions are `True`
provide list of tuples for `caselist` parameter:
1. boolean array or function taking series and returning boolean array
2. values when the boolean array is `True`
3. series of boolean `True` values with `Other` as replacement value


In [40]:
(make
 .case_when(caselist=[(make.isin(top5),make),
                      (make.isin(top10),'Top10'),
                      (pd.Series(True,index=make.index), 'Other')]))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

### Alternatively, with `.where`
the same thing can be achieved....

In [41]:
(make
 .where(make.isin(top5), 'Top10')
 .where(make.isin(top10), 'Other'))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: string[pyarrow]

The *cylinders* column has missing values. 

In [44]:
cyl = df.cylinders
(cyl
 .isna()
 .sum())


np.int64(206)

In [45]:
missing = cyl.isna()
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
34563     Tesla
34564     Tesla
34565     Tesla
34566     Tesla
34567     Tesla
Name: make, Length: 206, dtype: string[pyarrow]

*index operation* is performed with square brackets `[ ]`
If we index `.loc` with a boolean array, it returns rows where the boolean array is `True`
The `.fillna` method allows user to specify replacement value for any missing data
Operations return a new Series

In [46]:
cyl[cyl.isna()]

7138     <NA>
7139     <NA>
8143     <NA>
8144     <NA>
8146     <NA>
         ... 
34563    <NA>
34564    <NA>
34565    <NA>
34566    <NA>
34567    <NA>
Name: cylinders, Length: 206, dtype: int64[pyarrow]

In [48]:
cyl.fillna(0).loc[7136:7141]

7136    6
7137    6
7138    0
7139    0
7140    6
7141    6
Name: cylinders, dtype: int64[pyarrow]

In [50]:
temp = pd.Series([32,40,None,42,39,32], dtype='float[pyarrow]')
temp

0    32.0
1    40.0
2    <NA>
3    42.0
4    39.0
5    32.0
dtype: float[pyarrow]

### `.interpolate()` ordered data, like temperatures, to replace missing values

In [51]:
temp.interpolate()

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float[pyarrow]

### `.clip()` outliers in data
The first 447 entries in `city_mpg` range from 9 - 31. Trim these values to be between 5th (11.0) and 95th (27.0) quantile

In [52]:
(city_mpg
 .loc[:446]
 .clip(lower=city_mpg.quantile(.05),
       upper=city_mpg.quantile(.95)))

0      19
1      11
2      23
3      11
4      17
       ..
442    15
443    15
444    15
445    15
446    27
Name: city08, Length: 447, dtype: int64[pyarrow]

### Sorting values vs Sorting the index
the `.sort_values()` method will sort values in ascending order and also rearrange the index accordingly:

In [53]:
city_mpg.sort_values()

7901       6
21060      6
34557      6
35887      6
37161      6
        ... 
34563    138
34564    140
31256    150
32599    150
33423    150
Name: city08, Length: 41144, dtype: int64[pyarrow]

Because of index alignment, you can still do math operations on a sorted series

In [56]:
(city_mpg.sort_values() + highway_mpg) / 2

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: double[pyarrow]

In [60]:
rprint(city_mpg)
rprint(city_mpg.sort_values())
rprint(city_mpg.sort_values().sort_index())

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64[pyarrow]

7901       6
21060      6
34557      6
35887      6
37161      6
        ... 
34563    138
34564    140
31256    150
32599    150
33423    150
Name: city08, Length: 41144, dtype: int64[pyarrow]

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64[pyarrow]

The `.drop_duplicates()` method will remove values appearing more than once...you can keep the first, or last, or `False` to remove all duplicates, including the initial value

In [61]:
city_mpg.drop_duplicates()

0         19
1          9
2         23
3         10
4         17
        ... 
34364    127
34409    114
34564    140
34565    115
34566    104
Name: city08, Length: 105, dtype: int64[pyarrow]